## Предобработка

Установка необходимых библиотек.

In [ ]:
! pip update
! pip install numpy pandas opencv-python ultralytics

In [ ]:
import cv2
from ultralytics import YOLO
import pandas as pd
# При необходимости дописать

Функция для анализа видео.

In [ ]:
def detectLabels(videoPath):
    model = YOLO("yolov8m.pt")
    video = cv2.VideoCapture(videoPath)
    framesCount = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(video.get(cv2.CAP_PROP_FPS))
    print(f"Frames count: {framesCount}")\

    labelsDict = {"frame": [], "track_id": [], "x": [], "y": [], "h": [], "w": [], "label": []}

    for i in range(framesCount):
        ret, frame = video.read()
        
        res = model.track(frame, persist=True)

        if res[0].boxes.id is not None:  # Add this check
            cls = res[0].boxes.cls.int().cpu().tolist()
            boxes = res[0].boxes.xywh.cpu()
            track_ids = res[0].boxes.id.int().cpu().tolist()
        else:
            cls = []
            boxes = []
            track_ids = []

        for cl, box, track_id in zip(cls, boxes, track_ids):
            if cl != 0:
                continue

            x, y, w, h = box

            labelsDict['frame'].append(i)
            labelsDict['track_id'].append(track_id)
            labelsDict['x'].append(x.item())
            labelsDict['y'].append(y.item())
            labelsDict['w'].append(w.item())
            labelsDict['h'].append(h.item())
            labelsDict['label'].append(0)
    
    video.release()
    return (framesCount, fps, labelsDict)

In [ ]:
videoPath = "" # Введите путь до видеофайла
csvPath = "" # Введите путь до выходного файла
framesCount, fps, labelDict = detectLabels(videoPath)
df = pd.DataFrame(labelDict)
df.to_csv(csvPath)